In [1]:
import pandas as pd
import geopandas as gpd
import shapely

In [2]:
commuting_data_in = {}
commuting_data_out = {}
for i in range(1, 17):
    commuting_data_in[f"{i:02}"] = pd.read_excel(f"C:/Users/strobel/Projekte/open-data/BfA/krpend-{i:02}-0-202112-xlsx.xlsx", sheet_name="Einpendler Kreise", skiprows=6)
    commuting_data_out[f"{i:02}"] = pd.read_excel(f"C:/Users/strobel/Projekte/open-data/BfA/krpend-{i:02}-0-202112-xlsx.xlsx", sheet_name="Auspendler Kreise", skiprows=6)

In [3]:
workforce_stats = pd.read_excel("C:/Users/strobel/Projekte/open-data/BfA/gemband_dlk_0.xlsb", sheet_name="Gemeindedaten", skiprows=5)
workforce_stats = workforce_stats.rename(columns={"Unnamed: 0": "AGS", "Unnamed: 1": "Name", "Wohnort\ngleich\nArbeitsort": "InternalCommutes"})[[ "AGS", "Name", "InternalCommutes"]]
workforce_stats["AGS"] = workforce_stats["AGS"].dropna()
workforce_stats = workforce_stats[workforce_stats.AGS.apply(lambda x: 5 == len(str(x)) or str(x).endswith("000"))]
workforce_stats["AGS"] = workforce_stats["AGS"].apply(lambda x: str(x)[:5])
workforce_stats = workforce_stats.set_index("AGS")

In [4]:
def get_commuting_flows(for_area: str, direction: str):
    cols = ["Wohnort", "Arbeitsort", "Insgesamt"]
    if direction.lower() == "in":
        commuting_data = commuting_data_in[for_area[:2]][cols].copy()
        col_1 = 'Wohnort'
        col_2 = 'Arbeitsort'
    elif direction.lower() == "out":
        commuting_data = commuting_data_out[for_area[:2]][cols].copy()
        col_1 = 'Arbeitsort'
        col_2 = 'Wohnort'
    else:
        print(f"{direction} is not direction. Options: in, out")
        return
    
    # Fix excel formating
    commuting_data = commuting_data.fillna(method="ffill").replace("*", 0)
    # Filter Bundesländer and Bezirke
    commuting_data = commuting_data[commuting_data[col_1].apply(lambda x: len(str(x)) > 3)]
    # Get area
    commuting_data = commuting_data[commuting_data[col_2] == for_area]
    # Internal commuting
    commuting_data.loc[commuting_data.index.max()+1] = [for_area, for_area, workforce_stats.loc[for_area, "InternalCommutes"]]
    return commuting_data

def most_relevant(for_area: str):
    relevant_counties = set()
    for direction in ["in", "out"]:
        commuting_flows = get_commuting_flows(for_area, direction)
        # Get most relevant
        commuting_flows = commuting_flows.sort_values("Insgesamt", ascending=False)
        commuting_flows["cumsum_perc"] = commuting_flows.Insgesamt.cumsum()/ commuting_flows.Insgesamt.sum()
        commuting_flows = commuting_flows[commuting_flows.cumsum_perc <= 0.95]
        
        relevant_counties.update(commuting_flows["Arbeitsort"].to_list())
        relevant_counties.update(commuting_flows["Wohnort"].to_list())
    return relevant_counties

def make_od(relevant_counties):
    ods = pd.DataFrame(index=relevant_counties, columns=relevant_counties)
    for county in relevant_counties:
        for direction in ["in", "out"]:
            commuting_flows = get_commuting_flows(county, direction)
            commuting_flows = commuting_flows[commuting_flows.Wohnort.isin(relevant_counties) & commuting_flows.Arbeitsort.isin(relevant_counties)]
            # Pivot data
            od_county = commuting_flows.pivot(index=["Wohnort"], columns=["Arbeitsort"], values=["Insgesamt"])
            # Fix indices
            od_county.columns = od_county.columns.droplevel(0)
            od_county = od_county.rename_axis("Origin").rename_axis("Destination", axis=1)
            ods.loc[od_county.index, od_county.columns] = od_county
    return ods

In [5]:
relevant_counties = set()
relevant_counties.update(most_relevant('09462'))
relevant_counties.update(most_relevant('09472'))

In [6]:
od = make_od(relevant_counties)
od = od.fillna(0)

In [8]:
od["09462"]

11000      165
02000       57
09377      964
09461      225
09475     1056
09476      258
09562       57
09374      653
09474      240
09371      268
06412       27
09471      468
09564      295
09472    15510
09162      121
09479      631
09184       24
09462    19129
09362       36
09477     4182
09464      344
09478      330
09574      184
09363       73
Name: 09462, dtype: int64

In [8]:
krs = gpd.read_file("C:/Users/strobel/Projekte/esmregio/Daten/AdminGebiete/vg250_ebenen_0101/VG250_KRS.shp")
krs = krs[krs.AGS.isin(relevant_counties)][:-1] # Drop second hamburg (is it the harbor?)
krs = krs.to_crs(4326)

In [9]:
krs = krs.set_index("AGS")

In [10]:
od["geometry"] = krs.geometry

In [11]:
od = gpd.GeoDataFrame(od)

In [12]:
od.index.name = "origin"

In [13]:
od.to_file("tst.geojson")

In [40]:
out = od[["geometry"]].copy()
out["originActivity"] = "HOME"
out["destinationActivity"] = "WORK"
out["geometry"] = out.geometry.map(lambda geom: shapely.ops.transform(lambda x, y: (y, x), geom))
out["destinations"] = od.apply(lambda row: {c:row[c] for c in row.index.drop(["geometry"])}, axis=1)

In [42]:
out = out.set_geometry("geometry")

In [43]:
out.to_file("OD-Matrix.geojson")

In [41]:
def extract_coords(geom):
    poly = geom if type(geom) == shapely.geometry.polygon.Polygon else max(geom, key=lambda a: a.area)
    return  list(poly.exterior.coords)
od["geometry"] = od["geometry"].apply(extract_coords)

C:\Users\strobel\AppData\Local\Temp/ipykernel_9404/3414613903.py:2: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  poly = geom if type(geom) == shapely.geometry.polygon.Polygon else max(geom, key=lambda a: a.area)


In [44]:
od.to_csv("OD_matrix.csv", sep=";")

In [43]:
od

,09362,09564,09476,11000,09478,09475,09462,06412,09377,09479,...,09374,09474,09471,09184,09477,09562,09162,09574,02000,geometry
09362,45752.0,832.0,0,307.0,0.0,10,36.0,169.0,19,14.0,...,56.0,10.0,0.0,497.0,0.0,150.0,2039.0,50.0,153.0,"[(12.13170319505729, 49.076225536442514), (12...."
09564,401.0,149128.0,50,949.0,61.0,31,295.0,582.0,39,28.0,...,57.0,1242.0,288.0,852.0,52.0,11966.0,3831.0,7801.0,468.0,"[(11.001188608369826, 49.53981319043722), (11...."
09476,15.0,188.0,18442,54.0,1957.0,461,258.0,20.0,0,25.0,...,0.0,17.0,122.0,37.0,1196.0,68.0,149.0,19.0,11.0,"[(11.369787978557248, 50.52323971432081), (11...."
11000,177.0,1214.0,0,1216637.0,41.0,42,165.0,6555.0,0,32.0,...,19.0,39.0,63.0,1842.0,50.0,357.0,9500.0,80.0,9159.0,"[(13.480055493551298, 52.67464638244349), (13...."
09478,22.0,331.0,1083,118.0,17922.0,39,330.0,40.0,0,13.0,...,12.0,91.0,1172.0,32.0,1186.0,158.0,174.0,46.0,26.0,"[(11.034062949846902, 50.21864302123745), (11...."
09475,66.0,240.0,286,98.0,48.0,21749,1056.0,16.0,90,1527.0,...,26.0,17.0,39.0,66.0,1009.0,41.0,247.0,43.0,20.0,"[(11.921843069548048, 50.424093503683345), (11..."
09462,87.0,607.0,57,179.0,78.0,256,19129.0,83.0,241,108.0,...,274.0,56.0,60.0,99.0,1237.0,93.0,501.0,118.0,87.0,"[(11.58455018200977, 49.97593383017502), (11.5..."
06412,107.0,368.0,0,2550.0,0.0,0,27.0,212993.0,0,0.0,...,19.0,12.0,12.0,571.0,20.0,70.0,2971.0,21.0,1526.0,"[(8.711375837182405, 50.22625730179819), (8.71..."
09377,255.0,171.0,0,34.0,0.0,104,964.0,12.0,17678,3291.0,...,2693.0,0.0,21.0,56.0,46.0,59.0,154.0,40.0,0.0,"[(12.271271806863231, 50.0613597358564), (12.2..."
09479,147.0,181.0,10,44.0,20.0,1825,631.0,26.0,2029,19873.0,...,200.0,0.0,13.0,39.0,173.0,35.0,190.0,34.0,24.0,"[(12.088580974094405, 50.22563208304945), (12...."
